In [2]:
def change_list(lst):
    return [lst[-1]] + lst[1:-1] + [lst[0]]
change_list([1, 2, 3])

[3, 2, 1]

In [3]:
def swap_ends(l:list) -> list:
    return l[-1] + l.pop(0, -1) + l[0]

swap_ends([1, 2, 3])

TypeError: pop() takes at most 1 argument (2 given)

In [3]:
def reverseWords(s: str) -> str:
        return " ".join(s.split(" ")[::-1])

In [4]:
reverseWords("  hello world  ")

'  world hello  '

In [7]:
"  hello world  ".split()

['hello', 'world']